In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import numpy as np
import dask.dataframe as dd

In [8]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 23.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
import re
from gensim.models import TfidfModel, nmf
from gensim.corpora import Dictionary as Dict
from gensim.models.ldamulticore import LdaModel
from multiprocessing import Pool
from functools import partial
from gensim.models.nmf_pgd import solve_h

In [10]:
CHUNK_SIZE = 10000
TOPIC_NUM = 100
ALLFILES = ['logon.csv','device.csv', 'email.csv', 'file.csv', 'http.csv']
CONTENT_FILES = ['email.csv', 'file.csv', 'http.csv']

In [11]:
def check():
    if not output_dir.is_dir():
        os.makedirs(output_dir)
    assert (answers_dir.is_dir())
    assert (dataset_dir.is_dir())
    assert (main_answers_file.is_file())
    assert (output_dir.is_dir())

In [12]:
def count_file_lines(file):
    with open(file) as f:
        for count, _ in enumerate(f):
            pass
    return count

In [13]:
def collect_vocabulary(csv_name):
    result_set = set()

    for df in pd.read_csv(dataset_dir / f'{csv_name}.csv',
                          usecols=['date', 'user', 'content'], chunksize=CHUNK_SIZE):
        df['content'] = df['content'].str.lower().str.split()
        result_set = result_set.union(*map(set, df['content']))

    return result_set

In [14]:
def chunk_iterator(filename):
    for chunk in pd.read_csv(filename, chunksize=CHUNK_SIZE):
        for document in chunk['content'].str.lower().str.split().values:
            yield document


In [15]:
def tfidf_iterator(filenames, dictionary):
    for filename in filenames:
        for chunk in pd.read_csv(dataset_dir /filename, chunksize=CHUNK_SIZE):
            for document in chunk['content'].str.lower().str.split().values:
                yield dictionary.doc2bow(document)

In [16]:
def nmf_iterator(filenames, dictionary, tfidf):
    for filename in filenames:
        for chunk in pd.read_csv(dataset_dir /filename, chunksize=CHUNK_SIZE):
            for document in chunk['content'].str.lower().str.split().values:
                yield tfidf[dictionary.doc2bow(document)]

In [17]:
def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [18]:
def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

In [19]:
def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [21]:
def process_content(filename, chunk_size=CHUNK_SIZE):
    model = LdaModel.load((output_dir / 'lda_model.pkl').as_posix())
    temp_dict = Dict.load((output_dir / 'dict.pkl').as_posix())
    out_file = output_dir / (filename.stem + '_lda.csv')
    # touch one while not exist
    if not out_file.is_file():
        Path(out_file).touch()

    for chunk in pd.read_csv(filename, usecols=['id', 'content'], chunksize=chunk_size):
        chunk['content'] = chunk['content'].str.lower().str.split() \
            .apply(lambda doc: model[temp_dict.doc2bow(doc)])

        chunk.to_csv(out_file, mode='a')


In [22]:
def pre_process_logon(path):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S').dt.floor('D')
    df['day']=df['date'].dt.dayofweek
    
    self_pc = df \
        .groupby(['user', 'day', 'pc']).size().to_frame('count') \
        .reset_index().sort_values('count', ascending=False) \
        .drop_duplicates(subset=['user', 'day']) \
        .drop(columns=['count']).sort_values(['user', 'day']) \
        .groupby('user').pc.agg(pd.Series.mode).rename('self_pc')
    df = df.merge(self_pc.to_frame(), left_on='user', right_on='user')
    print("Done")
    print(df.head()) 
    #df['is_usual_pc'] = df['self_pc'] == df['pc']
   
    is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
    df['is_work_time'] = is_work_time
    
    df['subtype'] = df['activity']
    #df[['id', 'date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / 'logon_preprocessed.csv')
    df[['id', 'date', 'user', 'is_work_time', 'subtype']].to_csv(output_dir / 'logon_preprocessed.csv')
    return self_pc.to_frame()


In [23]:
def pre_process_device(path):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')
    df = df.merge(self_pc, left_on='user', right_on='user', )
    #df['is_usual_pc'] = df['self_pc'] == df['pc']

    is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
    df['is_work_time'] = is_work_time

    df['subtype'] = df['activity']
    #df[['id', 'date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(
    #    output_dir / f'device_preprocessed.csv')
    df[['id', 'date', 'user', 'is_work_time', 'subtype']].to_csv(
        output_dir / f'device_preprocessed.csv')

In [24]:
def pre_process_file(path):
    df = pd.read_csv(path, usecols=['id', 'date', 'user', 'pc', 'filename'])
    df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

    df = df.merge(self_pc, left_on='user', right_on='user', )
    #df['is_usual_pc'] = df['self_pc'] == df['pc']

    is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
    df['is_work_time'] = is_work_time

    file_extensions = df.filename.str[-4:]
    df['subtype'] = file_extensions
    #df[['id', 'date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(
    #    output_dir / f'file_preprocessed.csv')
    df[['id', 'date', 'user', 'is_work_time', 'subtype']].to_csv(
        output_dir / f'file_preprocessed.csv')

In [25]:
def pre_process_email(path):
    df = pd.read_csv(path, usecols=['id', 'date', 'user', 'pc', 'to', 'cc', 'bcc', 'from'])
    df = df.fillna('')
    to_concated = df[['to', 'cc', 'bcc']].apply(lambda x: ';'.join([x.to, x.cc, x.bcc]), axis=1)
    is_external_to = to_concated.apply(
        lambda x: any([re.match('^.+@(.+$)', e).group(1) != 'dtaa.com' for e in x.split(';') if e != '']))
    is_external = is_external_to | is_external_to
    df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

    df = df.merge(self_pc, left_on='user', right_on='user', )
    df['is_usual_pc'] = df['self_pc'] == df['pc']

    is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
    df['is_work_time'] = is_work_time

    df['subtype'] = is_external
    df[['id', 'date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(
        output_dir / f'email_preprocessed.csv')

In [26]:
def pre_process_http(path):

    # scenario 1
    scenario_1_http = [
        'actualkeylogger.com',
        'best-spy-soft.com',
        'dailykeylogger.com',
        'keylogpc.com',
        'refog.com',
        'relytec.com',
        'softactivity.com',
        'spectorsoft.com',
        'webwatchernow.com',
        'wellresearchedreviews.com',
        'wikileaks.org'
    ]

    # scenario 2
    scenario_2_http = [
        'careerbuilder.com',
        'craiglist.org',
        'indeed.com',
        'job-hunt.org',
        'jobhuntersbible.com',
        'linkedin.com',
        'monster.com',
        'simplyhired.com',
    ]

    
     #scenario 3
    scenario_3_http = [
        '4shared.com'
        'dropbox.com',
        'fileserve.com',
        'filefreak.com',
        'filestube.com',
        'megaupload.com',
        'thepiratebay.org'
    ]

    first_it = True
    mode = 'w'

    for http_df in pd.read_csv(path, chunksize=CHUNK_SIZE, usecols=['id','date', 'user', 'pc', 'url']):
        http_df['date'] = pd.to_datetime(http_df.date, format='%m/%d/%Y %H:%M:%S')

        site_names = http_df['url'].apply(lambda s: re.match('^https?://(www)?([0-9\-\w\.]+)?.+$', s).group(2))
        http_df['site_name'] = site_names

        http_df['subtype'] = 0
        http_df.loc[site_names.isin(scenario_1_http), 'subtype'] = 1
        http_df.loc[site_names.isin(scenario_2_http), 'subtype'] = 2
        http_df.loc[site_names.isin(scenario_3_http), 'subtype'] = 3

        http_df = http_df.merge(self_pc, left_on='user', right_on='user', )
        #http_df['is_usual_pc'] = http_df['self_pc'] == http_df['pc']

        is_work_time = (8 <= http_df.date.dt.hour) & (http_df.date.dt.hour < 17)
        http_df['is_work_time'] = is_work_time

        http_df.to_csv(output_dir / 'http_preprocessed.csv', header=first_it, index=False,
                       mode=mode, columns=['id', 'date', 'user', 'is_work_time', 'subtype', 'site_name'])
        first_it = False
        mode = 'a'

In [27]:
def merge_all_content():
    df_dict = Dict(chunk_iterator(dataset_dir / 'email.csv'))
    df_dict.add_documents(chunk_iterator(dataset_dir / 'file.csv'))
    df_dict.add_documents(chunk_iterator(dataset_dir / 'http.csv'))

    df_dict.save((output_dir / 'dict.pkl').as_posix())

In [28]:
def make_tfidf_model():
    tfidf_model = TfidfModel(
        tfidf_iterator(CONTENT_FILES, Dict.load((output_dir / 'dict.pkl').as_posix())))

    tfidf_model.save((output_dir / 'tfidf_model.pkl').as_posix())


In [29]:
def make_nmf_model():
    tfidf_model = TfidfModel.load((output_dir / 'tfidf_model.pkl').as_posix())
    nmf_model = nmf.Nmf(
        nmf_iterator(CONTENT_FILES, Dict.load((output_dir / 'dict.pkl').as_posix()),
                     tfidf_model), num_topics=TOPIC_NUM)
    nmf_model.save((output_dir / 'nmf_model.pkl').as_posix())

In [30]:
def make_lda_model():
    tfidf_model = TfidfModel.load((output_dir / 'tfidf_model.pkl').as_posix())
    lda_model = LdaModel(
        nmf_iterator(CONTENT_FILES, Dict.load((output_dir / 'dict.pkl').as_posix()),
                     tfidf_model), num_topics=TOPIC_NUM)
    lda_model.save((output_dir / 'lda_model.pkl').as_posix())

In [ ]:
if __name__ == "__main__":
    #if len(sys.argv) != 3:
    #    print("Usage:", sys.argv[0], "[answer directory] [dataset directory]")
    #    print("Warning: may cost over 30 hours to run all steps for the dataset over 20G")
    #    print("hint: comment out the pre_runned process")

    #print("Start to process dataset ver", sys.argv[2].split('/')[-1])
    answers_dir = Path("./answers")
    dataset_dir = Path("./dataset")
    main_answers_file = answers_dir / "insiders.csv"
    output_dir = Path('./_output/')

    if not os.path.isdir(output_dir):  # case of no _output directory
        os.mkdir(output_dir)

    self_pc = pre_process_logon(dataset_dir / '/kaggle/input/cert-insider-threat-detection-research/logon.csv')
    print("logon processed")

    pre_process_device(dataset_dir / '/kaggle/input/cert-insider-threat-detection-research/device.csv')
    print("device processed")

    pre_process_file(dataset_dir / '/kaggle/input/cert-insider-threat-detection-research/file.csv')
    print("file processed")

    #pre_process_email(dataset_dir / '/kaggle/input/cert-insider-threat-detection-research/email.csv')
    #print("email processed")

    pre_process_http(dataset_dir / '/kaggle/input/cert-insider-threat-detection-research/http.csv')
    print("http processed")

    merge_all_content()
    print("all content file merged and saved")
    make_tfidf_model()
    print("tfidf model saved")
    make_nmf_model()
    print("nmf model saved")
    make_lda_model()
    print("lda model saved")

    #pre process all content files
    for file in CONTENT_FILES:
        process_content(dataset_dir / file)
        print(file, "content processed")